In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sbe_vallib import Validation, BinaryScorer, SupervisedSampler

## Dataset

In [3]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

X, y = make_classification(n_samples=1000, n_classes=2,
                               n_features=5, n_informative=3, n_redundant=0,
                               random_state=0)

X = pd.DataFrame(X)
y = pd.Series(y)

X_train, X_test,  y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)


## Model

In [4]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression().fit(X_train, y_train)

## Sampler Scorer

In [5]:
from sbe_vallib import SupervisedSampler, BinaryScorer

sampler = SupervisedSampler(train={'X': X_train, 'y_true': y_train, 'y_pred': model.predict_proba(X_train)},
                        oos = {'X': X_test, 'y_true': y_test, 'y_pred': model.predict_proba(X_test)})

scorer = BinaryScorer()

In [10]:
def custom_test(model, scorer, sampler, a = 10, **kwargs):
    metrics = []
    for i in range(a):
        sampler.set_state(i)
        train = sampler.train
        oos = sampler.oos
        model.fit(X=train["X"], y=train["y_true"])
        y_pred = model.predict_proba(oos['X'])
        metrics.append(scorer.calc_metrics(oos['y_true'], y_pred))
    
    return {
        "semaphore": "grey",
        "result_dict": {'mean_f1': np.mean([i['f1_score'] for i in metrics])},
        "result_dataframes": [pd.DataFrame(metrics)],
        "result_plots": [],
    }


custom_tests = {
    "test_0_0": {"block": "model_stability", "callable": custom_test, "params": {}},
    "test_0_0_1": {
        "block": "model_stability",
        "callable": custom_test,
        "params": {"a": 100},
    },
}


validor = Validation(model, sampler, scorer, custom_tests=custom_tests,
                         pipeline='../src/sbe_vallib/table/pipelines/Config_31.xlsx')
res = validor.validate()
res.keys()

/Users/azatsultanov/Programming/vallib/vallib_env/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


dict_keys(['test_factor_psi', 'test_ci', 'test_0_0', 'test_0_0_1'])

In [11]:
res['test_0_0_1']['result_dataframes'][0]

,accuracy_score,precision_score,recall_score,f1_score,gini
0,0.713333,0.723684,0.714286,0.718954,0.594912
1,0.713333,0.709091,0.754839,0.731250,0.573838
2,0.766667,0.785714,0.733333,0.758621,0.669156
3,0.733333,0.742857,0.702703,0.722222,0.620377
4,0.713333,0.750000,0.713415,0.731250,0.552636
...,...,...,...,...,...
95,0.726667,0.729167,0.709459,0.719178,0.584015
96,0.723333,0.737179,0.732484,0.734824,0.613737
97,0.710000,0.693333,0.717241,0.705085,0.570990
98,0.676667,0.691729,0.621622,0.654804,0.480263
